In [1]:
import copy

import numpy as np
from matplotlib import pyplot as plt
#Import the needed opentps.core packages
import time
import os
import logging
import math
import numpy as np
from matplotlib import pyplot as plt
import sys
from opentps.core.io.dicomIO import writeRTPlan
from opentps.core.processing.planOptimization.tools import evaluateClinical
from opentps.core.data.images import CTImage
from opentps.core.data.images import ROIMask
from opentps.core.data.plan import ObjectivesList
from opentps.core.data.plan import PlanDesign
from opentps.core.data import DVH
from opentps.core.data import Patient
from opentps.core.data.plan import FidObjective
from opentps.core.io.scannerReader import readScanner
from opentps.core.io.serializedObjectIO import saveRTPlan, loadRTPlan
from opentps.core.processing.doseCalculation.doseCalculationConfig import DoseCalculationConfig
from opentps.core.processing.doseCalculation.mcsquareDoseCalculator import MCsquareDoseCalculator
from opentps.core.processing.imageProcessing.resampler3D import resampleImage3DOnImage3D, resampleImage3D
from opentps.core.processing.C_libraries.libRayTracing_wrapper import WET_raytracing
from opentps.core.processing.planOptimization.planOptimization import IMPTPlanOptimizer
from opentps.core.data.plan import PlanDesign
from opentps.core.data import DVH
from opentps.core.io import mcsquareIO
from opentps.core.io.mcsquareIO import _dicomIsocenterToMCsquare
from opentps.core.io.scannerReader import readScanner
from opentps.core.processing.doseCalculation.doseCalculationConfig import DoseCalculationConfig
from opentps.core.processing.doseCalculation.mcsquareDoseCalculator import MCsquareDoseCalculator
from opentps.core.io.dataLoader import readData
from opentps.core.data.plan import ObjectivesList
from opentps.core.data.plan import FidObjective
from opentps.core.io.serializedObjectIO import saveBeamlets, saveRTPlan, loadBeamlets, loadRTPlan

############################################### 0604 TG119 TEST ##############################################

ctImagePath = "E:\opentps-v1.1.1\opentps_core\opentps\core\examples\planOptimization\WaterBox_RStest"
data = readData(ctImagePath)
#print(data[0])
# RD RT RS CT
# RD RD RD RD RD RP RS CT


24/07/2025 11:25:29 AM - root - INFO - Loading logging configuration: E:\opentps-v1.1.1\opentps_core\opentps\core\config\logger\logging_config.json
24/07/2025 11:25:29 AM - opentps.core._loggingConfig - INFO - Log level set: INFO


d:\Anaconda\envs\OpenTPS_v1\lib\site-packages\cupyx\jit\_interface.py:173: FutureWarning: cupyx.jit.rawkernel is experimental. The interface can change in the future.
  cupy._util.experimental('cupyx.jit.rawkernel')


In [7]:
ctCalibration = readScanner(DoseCalculationConfig().scannerFolder)
bdl = mcsquareIO.readBDL("E:\opentps-v1.1.1\opentps_core\opentps\core\processing\doseCalculation\MCsquare\BDL\BDL_default_DN_RangeShifter.txt")

In [13]:
ct.gridSize

array([512, 512, 483])

In [25]:
data[0].print_ROINames()
ct=data[1]
body=data[0].getContourByName("BODY")
body_mask=body.getBinaryMask(gridSize=ct.gridSize, origin=ct.origin, spacing=ct.spacing)
# body_mask_roi=body_mask.getBinaryContourMask()


RT Struct UID: 1.2.246.352.221.5624734378809543997.6995646488669454524
  [0]  zD-102%
  [1]  CTV
  [2]  R10M08FS20_NEW
  [3]  Target_Line10cm
  [4]  Target_line
  [5]  R10M8FS20
  [6]  R10M8FS10
  [7]  FS10
  [8]  R10M6FS10
  [9]  R30M10FS10
  [10]  R20M10FS10
  [11]  R10M10FS10
  [12]  BODY


In [29]:
body_mask.imageArray.shape

(512, 512, 483)

In [ ]:
# body_mask_resampled = resampleImage3D(body_mask,spacing=(2.5,2.5,2.5))
# body_mask_resampled.imageArray.shape
# body_mask_resampled_transposed = np.transpose(body_mask_resampled.imageArray, (2, 1, 0))

In [28]:
np.save("waterphatom_body_mask.npy",body_mask.imageArray)

In [7]:
plan=data[-3]
rt_struct = data[-2]
ct = data[-1]
rt_struct.print_ROINames()
print("CT spacing=",ct.spacing)
print("CT grid size",ct.gridSize)
target_name = "CTV"



RT Struct UID: 1.2.246.352.221.5624734378809543997.6995646488669454524
  [0]  zD-102%
  [1]  CTV
  [2]  R10M08FS20_NEW
  [3]  Target_Line10cm
  [4]  Target_line
  [5]  R10M8FS20
  [6]  R10M8FS10
  [7]  FS10
  [8]  R10M6FS10
  [9]  R30M10FS10
  [10]  R20M10FS10
  [11]  R10M10FS10
  [12]  BODY
CT spacing= [1.40625 1.40625 1.     ]
CT grid size [512 800 483]


In [ ]:
#MCsquare simulation
mc2 = MCsquareDoseCalculator()
mc2.beamModel = bdl
mc2.ctCalibration = ctCalibration
mc2.nbPrimaries = 1e7
doseImage = mc2.computeDose(ct, plan)

27/06/2025 03:44:59 PM - opentps.core.processing.doseCalculation.mcsquareDoseCalculator - INFO - Prepare MCsquare Dose calculation
27/06/2025 03:44:59 PM - opentps.core.io.mhdIO - INFO - Write MHD file: C:\Users\gcyyy\openTPS_workspace\Simulations\MCsquare_simulation\CT.mhd
27/06/2025 03:45:00 PM - opentps.core.io.mcsquareIO - INFO - Write plan: C:\Users\gcyyy\openTPS_workspace\Simulations\MCsquare_simulation\PlanPencil.txt
27/06/2025 03:45:04 PM - opentps.core.processing.doseCalculation.mcsquareDoseCalculator - INFO - Start MCsquare simulation


In [ ]:
from ipywidgets import interact, IntSlider
import matplotlib.pyplot as plt
import numpy as np
def plot_Zslice(z_index):
    #plt.clf()
    plt.figure(figsize=(8, 6))
    x_size=doseImage.imageArray.shape[0]
    y_size=doseImage.imageArray.shape[1]
    X=np.linspace(0,x_size-1,x_size)
    Y=np.linspace(0,y_size-1,y_size)
    X, Y = np.meshgrid(X, Y)
    #plt.contourf(X, Y, covered_count[:,:,z_index], cmap='jet',levels=10,vmin=0,vmax=np.max(covered_count),alpha=0.6)
    plt.imshow(doseImage.imageArray[:,:,z_index].transpose(1,0), cmap='jet', interpolation='nearest', alpha=0.6)
    plt.colorbar()
    #绘制边界
    #plt.contour(X, Y, expanded_image[:,:,z_index], colors='k',linewidths=2,levels=[0.5])
    plt.plot()
    plt.show()   
interact(plot_Zslice, z_index=IntSlider(min=0, max=doseImage.imageArray.shape[2]-1, step=1))     

In [ ]:
doseImage.imageArray.shape

In [ ]:
from opentps.core.io.dicomIO import writeRTDose
output_folder = os.getcwd()
writeRTDose(plan,output_folder,"opentps_SFRT_250116.dcm")

# import pydicom
# import datetime
# # Don't delete it
# # ++++Don't delete UIDs to build the simple study+++++++++++++++++++
# studyInstanceUID = pydicom.uid.generate_uid()
# doseSeriesInstanceUID = pydicom.uid.generate_uid()
# planSeriesInstanceUID = pydicom.uid.generate_uid()
# ctSeriesInstanceUID =  pydicom.uid.generate_uid()
# frameOfReferenceUID = pydicom.uid.generate_uid()
# # structSeriesInstanceUID = pydicom.uid.generate_uid()
# dt = datetime.datetime.now()
# # CT
# patient = Patient()
# patient.name = 'Simple_Patient'
# Patient.id = 'Simple_Patient'
# Patient.birthDate = dt.strftime('%Y%m%d')
# patient.sex = ""

# doseImage.referencePlan = plan
# doseImage.referenceCT = ct
# doseImage.patient = patient
# doseImage.studyInstanceUID = studyInstanceUID
# doseImage.frameOfReferenceUID = frameOfReferenceUID 
# doseImage.sopClassUID = '1.2.840.10008.5.1.4.1.1.481.2'
# doseImage.mediaStorageSOPClassUID = '1.2.840.10008.5.1.4.1.1.481.2'
# doseImage.sopInstanceUID = pydicom.uid.generate_uid()
# doseImage.studyTime = dt.strftime('%H%M%S.%f')
# doseImage.studyDate = dt.strftime('%Y%m%d')

# writeRTDose(doseImage,output_folder)

09/04/2025 02:45:43 PM - opentps.core.io.dicomIO - INFO - Export dicom RTDOSE: RD1.2.826.0.1.3680043.8.498.13788487360005424824957731664625185789.dcm in e:\opentps-v1.1.1\opentps_core\opentps\core\examples\planOptimization


d:\Anaconda\envs\OpenTPS_new\lib\site-packages\pydicom\dataset.py:2231: UserWarning: Camel case attribute 'SoftwareVersion' used which is not in the element keyword data dictionary
  warnings.warn(msg)
d:\Anaconda\envs\OpenTPS_new\lib\site-packages\pydicom\dataset.py:2231: UserWarning: Camel case attribute 'OperatorName' used which is not in the element keyword data dictionary
  warnings.warn(msg)
d:\Anaconda\envs\OpenTPS_new\lib\site-packages\pydicom\dataset.py:2231: UserWarning: Camel case attribute 'Width' used which is not in the element keyword data dictionary
  warnings.warn(msg)
d:\Anaconda\envs\OpenTPS_new\lib\site-packages\pydicom\valuerep.py:443: UserWarning: A value of type 'int32' cannot be assigned to a tag with VR US.
  warnings.warn(msg)
d:\Anaconda\envs\OpenTPS_new\lib\site-packages\pydicom\dataset.py:2231: UserWarning: Camel case attribute 'Height' used which is not in the element keyword data dictionary
  warnings.warn(msg)
d:\Anaconda\envs\OpenTPS_new\lib\site-packag